In [1]:
# Import libraries
import os
import glob
from osgeo import gdal
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage
import pandas as pd
import datetime as dt
import time

In [2]:
# Set input directory, and change working directory
inDir = 'D:/tyler/Documents/SCHOOL/SPRING_2021/project-code/Test Sets/MOD13Q1.006__250m_16_days_NDVI'  # IMPORTANT: Update to reflect directory on your OS
os.chdir(inDir)                                                               # Change to working directory
outDir = os.path.normpath(os.path.split(inDir)[0] + os.sep + 'output') + '\\' # Create and set output directory
if not os.path.exists(outDir): os.makedirs(outDir)

In [3]:
EVIFiles = glob.glob('MOD13Q1.006__250m_16_days_NDVI_**.tif') # Search for and create a list of EVI files

In [4]:
print(len(EVIFiles))

47


In [6]:
start_time = time.time() # Start timing

for i in range(len(EVIFiles)):
    EVI = gdal.Open(EVIFiles[i])                    # Read file in, starting with MOD13Q1 version 6
    EVIBand = EVI.GetRasterBand(1)                  # Read the band (layer)
    EVIData = EVIBand.ReadAsArray().astype('float') # Import band as an array with type float
    
    # File name metadata:
    productId = EVIFiles[i].split('_')[0]                                          # First: product name
    layerId = EVIFiles[i].split(productId + '_')[1].split('_doy')[0]               # Second: layer name
    yeardoy = EVIFiles[i].split(layerId+'_doy')[1].split('_aid')[0]                # Third: date
    aid = EVIFiles[i].split(yeardoy+'_')[1].split('.tif')[0]                       # Fourth: unique ROI identifier (aid)
    date = dt.datetime.strptime(yeardoy, '%Y%j').strftime('%m/%d/%Y')              # Convert YYYYDDD to MM/DD/YYYY
    
    # File Metadata
    EVI_meta = EVI.GetMetadata()                   # Store metadata in dictionary
    rows, cols = EVI.RasterYSize, EVI.RasterXSize  # Number of rows,columns

    # Projection information
    geotransform = EVI.GetGeoTransform()
    proj= EVI.GetProjection() 

    # Band metadata
    EVIFill = EVIBand.GetNoDataValue()            # Returns fill value
    EVIStats = EVIBand.GetStatistics(True, True)  # returns min, max, mean, and standard deviation
    EVI = None                                    # Close the GeoTIFF file
    
    scaleFactor = float(EVI_meta['scale_factor'])  # Search the metadata dictionary for the scale factor 
    units = EVI_meta['units']                      # Search the metadata dictionary for the units
    EVIData[EVIData == EVIFill] = np.nan           # Set the fill value equal to NaN for the array
    EVIScaled = EVIData * scaleFactor              # Apply the scale factor using simple multiplication

    # Generate statistics on the scaled data
    EVIStats_sc = [np.nanmin(EVIScaled), np.nanmax(EVIScaled), np.nanmean(EVIScaled), np.nanstd(EVIScaled)] # Create a list of stats
    
    lut = glob.glob('**lookup.csv')                                        # Search for look up table 
    qualityFiles =glob.glob('MOD13Q1.006__250m_16_days_VI_Quality**.tif')  # Search the directory for the associated quality .tifs
    quality = gdal.Open(qualityFiles[i])                                   # Open the first quality file
    qualityData = quality.GetRasterBand(1).ReadAsArray()                   # Read in as an array
    quality = None                                                         # Close the quality file
    
    v6_QA_lut = pd.read_csv(lut[0])     # Read in the lut
    
    # Include good quality based on MODLAND
    v6_QA_lut = v6_QA_lut[v6_QA_lut['MODLAND'].isin(['VI produced with good quality', 'VI produced, but check other QA'])]

    # Exclude lower quality VI usefulness
    VIU =["Lowest quality","Quality so low that it is not useful","L1B data faulty","Not useful for any other reason/not processed"]
    v6_QA_lut = v6_QA_lut[~v6_QA_lut['VI Usefulness'].isin(VIU)]

    v6_QA_lut = v6_QA_lut[v6_QA_lut['Aerosol Quantity'].isin(['Low','Average'])]   # Include low or average aerosol
    v6_QA_lut = v6_QA_lut[v6_QA_lut['Adjacent cloud detected'] == 'No' ]           # Include where adjacent cloud not detected
    v6_QA_lut = v6_QA_lut[v6_QA_lut['Mixed Clouds'] == 'No' ]                      # Include where mixed clouds not detected
    v6_QA_lut = v6_QA_lut[v6_QA_lut['Possible shadow'] == 'No' ]                   # Include where possible shadow not detected
    
    goodQuality = list(v6_QA_lut['Value']) # Retrieve list of possible QA values from the quality dataframe
    EVI_masked = np.ma.MaskedArray(EVIScaled, np.in1d(qualityData, goodQuality, invert = True))    # Apply QA mask to the EVI data
    
    masked_compressed = EVI_masked.compressed() # Accessing only valid entries from the masked array

    # Write all of the quality pixel values to a CSV file with the same name as the original .tif file
    with open('D:/tyler/Documents/SCHOOL/SPRING_2021/project-code/Test Sets/MOD13Q1.006__250m_16_days_NDVI_CSV/' + f'{productId}{layerId}_{yeardoy}.csv', 'w') as file:
        for j in range(len(masked_compressed)-1):
            file.write(str(masked_compressed[j]) + ',\n')
        file.write(str(masked_compressed[-1]) + '\n')

# End timing
print(f"Time in seconds: {time.time() - start_time}")

Time in seconds: 1256.9209253787994
